In [2]:

# Import Dependencies
%matplotlib inline

# Start Python Imports
import math, time, random, datetime
from math import sqrt
# Data Manipulation
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Visualization 
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-whitegrid')

# Preprocessing
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, label_binarize

# Machine learning
from sklearn.model_selection import train_test_split
from sklearn import model_selection, tree, preprocessing, metrics, linear_model
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb
import re
# Let's be rebels and ignore warnings for now
import warnings
warnings.filterwarnings('ignore')
regex = re.compile(r"\[|\]|<", re.IGNORECASE)


In [3]:
my_data = pd.read_csv("Downloads/ML/tcd ml 2019-20 income prediction training (with labels).csv")
my_prediction_data = pd.read_csv("Downloads/ML/tcd ml 2019-20 income prediction training (with labels).csv")
my_test_data = pd.read_csv("Downloads/ML/tcd ml 2019-20 income prediction test (without labels).csv")
predicted_income = pd.read_csv("Downloads/ML/tcd ml 2019-20 income prediction submission file example (random solutions).csv")

In [4]:
my_data.head()

,Instance,Year of Record,Gender,Age,Country,Size of City,Profession,University Degree,Wears Glasses,Hair Color,Body Height [cm],Income in EUR
0,1,1997.0,0,41.0,Belarus,1239930,steel workers,Bachelor,0,Blond,193,61031.94416
1,2,1996.0,other,41.0,Singapore,1603504,safe event coordinator,Master,0,Black,186,91001.32764
2,3,2018.0,other,28.0,Norway,1298017,receivables/payables analyst,PhD,1,Brown,170,157982.17670
3,4,2006.0,other,33.0,Cuba,751903,fleet assistant,No,1,Black,171,45993.75793
4,5,2010.0,female,46.0,United Arab Emirates,95389,lead trainer,0,0,Blond,188,38022.16217


In [5]:
my_test_data.head()

,Instance,Year of Record,Gender,Age,Country,Size of City,Profession,University Degree,Wears Glasses,Hair Color,Body Height [cm],Income
0,111994,1992.0,other,21.0,Honduras,391652,senior project analyst,Master,1,Brown,153,NaN
1,111995,1986.0,other,34.0,Kyrgyzstan,33653,greeter,Bachelor,0,Black,163,NaN
2,111996,1994.0,unknown,53.0,Portugal,34765,liaison,Bachelor,1,Blond,153,NaN
3,111997,1984.0,0,29.0,Uruguay,1494132,occupational therapist,No,0,Black,154,NaN
4,111998,2007.0,other,17.0,Serbia,120661,portfolio manager,No,0,Red,191,NaN


In [6]:
def missing(df, columns):
    missing = {}
    print("missing vals")
    length = len(df)
    for i in columns:
        total = df[i].value_counts().sum()
        missing[i] = length - total
    print(missing)
        
missing(my_data, columns=my_data.columns)  

missing vals
{'Instance': 0, 'Year of Record': 441, 'Gender': 7432, 'Age': 494, 'Country': 0, 'Size of City': 0, 'Profession': 322, 'University Degree': 7370, 'Wears Glasses': 0, 'Hair Color': 7242, 'Body Height [cm]': 0, 'Income in EUR': 0}


In [7]:
my_data['Age'].value_counts()

15.0     2793
22.0     2735
18.0     2733
21.0     2730
24.0     2724
17.0     2718
16.0     2702
19.0     2701
29.0     2698
20.0     2697
23.0     2666
27.0     2665
30.0     2629
26.0     2620
25.0     2616
28.0     2593
34.0     2575
32.0     2569
35.0     2529
31.0     2501
36.0     2439
33.0     2418
37.0     2417
40.0     2322
39.0     2307
41.0     2289
38.0     2267
42.0     2135
44.0     2113
43.0     2068
         ... 
82.0      111
85.0       98
84.0       97
86.0       83
87.0       79
88.0       54
89.0       52
90.0       42
91.0       38
92.0       37
94.0       29
93.0       25
100.0      13
97.0       12
99.0       12
95.0       11
96.0       11
101.0       9
98.0        6
105.0       5
102.0       5
103.0       5
108.0       4
107.0       4
104.0       4
110.0       4
111.0       2
109.0       1
115.0       1
106.0       1
Name: Age, Length: 99, dtype: int64

In [8]:
my_data['Age'].fillna(my_data['Age'].mean(), inplace=True)
my_test_data['Age'].fillna(my_test_data['Age'].mean(), inplace=True)

In [9]:
my_data['Age'].isnull().sum()

0

In [10]:
my_data['Gender'].value_counts()

male       42758
female     27170
other      27060
unknown     6849
0            724
Name: Gender, dtype: int64

In [11]:
my_data['Gender'].fillna('unknown', inplace=True)
my_test_data['Gender'].fillna('unknown', inplace=True)

In [12]:
my_data['Gender'].isnull().sum()

0

In [13]:
my_data = my_data[my_data['Gender'] != '0']

In [14]:
my_data['Country'].isnull().sum()

0

In [15]:
my_data['University Degree'].value_counts()

Bachelor    42895
No          26815
Master      26771
PhD          6777
0             695
Name: University Degree, dtype: int64

In [16]:
my_data['University Degree'].fillna(my_data['University Degree'].mode()[0], inplace=True)
my_test_data['University Degree'].fillna(my_test_data['University Degree'].mode()[0], inplace=True)

In [17]:
my_data['University Degree'].isnull().sum()

0

In [18]:
my_data['Year of Record'].fillna(my_data['Year of Record'].mode()[0], inplace=True)
my_test_data['Year of Record'].fillna(my_test_data['Year of Record'].mode()[0], inplace=True)

In [19]:
my_data['Year of Record'].isnull().sum()

0

In [20]:
missing(my_data, columns=my_data.columns)

missing vals
{'Instance': 0, 'Year of Record': 0, 'Gender': 0, 'Age': 0, 'Country': 0, 'Size of City': 0, 'Profession': 319, 'University Degree': 0, 'Wears Glasses': 0, 'Hair Color': 7194, 'Body Height [cm]': 0, 'Income in EUR': 0}


In [21]:
my_data['Profession'].fillna(my_data['Profession'].mode()[0], inplace=True)

In [22]:
my_data['Hair Color'].fillna(my_data['Hair Color'].mode()[0], inplace=True)
my_test_data['Hair Color'].fillna(my_test_data['Hair Color'].mode()[0], inplace=True)

In [23]:
my_data['Profession'].fillna(my_data['Profession'].mode()[0], inplace=True)
my_test_data['Profession'].fillna(my_test_data['Profession'].mode()[0], inplace=True)

In [24]:
my_data['Hair Color'].isnull().sum()

0

In [25]:
my_data['Profession'].isnull().sum()

0

In [26]:
missing(my_test_data, columns=my_test_data.columns)

missing vals
{'Instance': 0, 'Year of Record': 0, 'Gender': 0, 'Age': 0, 'Country': 0, 'Size of City': 0, 'Profession': 0, 'University Degree': 0, 'Wears Glasses': 0, 'Hair Color': 0, 'Body Height [cm]': 0, 'Income': 73230}


In [27]:
missing(my_data, columns=my_data.columns)

missing vals
{'Instance': 0, 'Year of Record': 0, 'Gender': 0, 'Age': 0, 'Country': 0, 'Size of City': 0, 'Profession': 0, 'University Degree': 0, 'Wears Glasses': 0, 'Hair Color': 0, 'Body Height [cm]': 0, 'Income in EUR': 0}


In [28]:
df_continous = pd.DataFrame()

In [29]:
df_continous['Year of Record'] = my_data['Year of Record']
df_continous['Age'] = my_data['Age']
df_continous['Gender'] = my_data['Gender']
df_continous['Country'] = my_data['Country']
df_continous['Size of City'] = my_data['Size of City']
df_continous['Profession'] = my_data['Profession']
df_continous['University Degree'] = my_data['University Degree']
df_continous['Wears Glasses'] = my_data['Wears Glasses']
df_continous['Body Height [cm]'] = my_data['Body Height [cm]']
df_continous['Income in EUR'] = my_data['Income in EUR']
df_continous['Hair Color'] = my_data['Hair Color']

In [30]:
missing(df_continous, columns=df_continous.columns) 

missing vals
{'Year of Record': 0, 'Age': 0, 'Gender': 0, 'Country': 0, 'Size of City': 0, 'Profession': 0, 'University Degree': 0, 'Wears Glasses': 0, 'Body Height [cm]': 0, 'Income in EUR': 0, 'Hair Color': 0}


In [31]:
df_continous.head()

,Year of Record,Age,Gender,Country,Size of City,Profession,University Degree,Wears Glasses,Body Height [cm],Income in EUR,Hair Color
1,1996.0,41.0,other,Singapore,1603504,safe event coordinator,Master,0,186,91001.32764,Black
2,2018.0,28.0,other,Norway,1298017,receivables/payables analyst,PhD,1,170,157982.17670,Brown
3,2006.0,33.0,other,Cuba,751903,fleet assistant,No,1,171,45993.75793,Black
4,2010.0,46.0,female,United Arab Emirates,95389,lead trainer,0,0,188,38022.16217,Blond
5,1991.0,60.0,male,Liberia,56080,senior security analyst,Bachelor,0,181,125809.99030,Blond


In [32]:
my_data['train'] = 1
my_test_data['train'] = 0

In [33]:
final_df = pd.concat([my_data, my_test_data])

In [34]:
# One-hot encode non binary variables
df_profession_oneHot = pd.get_dummies(final_df['Profession'],
                                     prefix='pro')
df_gender_oneHot = pd.get_dummies(final_df['Gender'],
                                     prefix='gen')
df_country_oneHot = pd.get_dummies(final_df['Country'],
                                     prefix='cou')
df_ud_oneHot = pd.get_dummies(final_df['University Degree'],
                                     prefix='ud')
df_hc_oneHot = pd.get_dummies(final_df['Hair Color'],
                                     prefix='h_c')
#print(df_profession_oneHot)

In [35]:
final_df = pd.concat([final_df,
                              df_profession_oneHot,df_gender_oneHot,df_country_oneHot,df_ud_oneHot,df_hc_oneHot
                             ], axis=1 )

In [36]:
train_df = final_df[final_df['train'] == 1]
test_df = final_df[final_df['train'] == 0]

In [37]:
print(train_df.shape)
print(test_df.shape)

(111269, 1547)
(73230, 1547)


In [38]:
train_df.drop(['train','Profession','Gender','Country','University Degree','Hair Color'], axis=1,inplace=True)
test_df.drop(['train','Profession','Gender','Country','University Degree','Hair Color'], axis=1,inplace=True)

In [39]:
train_df.set_index('Instance', inplace=True)

In [40]:
test_df.set_index('Instance', inplace=True)

In [41]:
test_df.head()

,Age,Body Height [cm],Income,Income in EUR,Size of City,Wears Glasses,Year of Record,pro_.net developer,pro_.net software developer,pro_Accountant,...,ud_Bachelor,ud_Master,ud_No,ud_PhD,h_c_0,h_c_Black,h_c_Blond,h_c_Brown,h_c_Red,h_c_Unknown
Instance,,,,,,,,,,,,,,,,,,,,,
111994,21.0,153,NaN,NaN,391652,1,1992.0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
111995,34.0,163,NaN,NaN,33653,0,1986.0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
111996,53.0,153,NaN,NaN,34765,1,1994.0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
111997,29.0,154,NaN,NaN,1494132,0,1984.0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
111998,17.0,191,NaN,NaN,120661,0,2007.0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [42]:
train_df.head()

,Age,Body Height [cm],Income,Income in EUR,Size of City,Wears Glasses,Year of Record,pro_.net developer,pro_.net software developer,pro_Accountant,...,ud_Bachelor,ud_Master,ud_No,ud_PhD,h_c_0,h_c_Black,h_c_Blond,h_c_Brown,h_c_Red,h_c_Unknown
Instance,,,,,,,,,,,,,,,,,,,,,
2,41.0,186,NaN,91001.32764,1603504,0,1996.0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
3,28.0,170,NaN,157982.17670,1298017,1,2018.0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
4,33.0,171,NaN,45993.75793,751903,1,2006.0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
5,46.0,188,NaN,38022.16217,95389,0,2010.0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
6,60.0,181,NaN,125809.99030,56080,0,1991.0,0,0,0,...,1,0,0,0,0,0,1,0,0,0


In [43]:
test_df.drop(['Income','Income in EUR'], axis=1,inplace=True)

In [44]:
train_df.drop(['Income'],axis=1,inplace=True)

In [45]:
test_df.head()

,Age,Body Height [cm],Size of City,Wears Glasses,Year of Record,pro_.net developer,pro_.net software developer,pro_Accountant,pro_Accounts Assistant,pro_Accounts Clerk,...,ud_Bachelor,ud_Master,ud_No,ud_PhD,h_c_0,h_c_Black,h_c_Blond,h_c_Brown,h_c_Red,h_c_Unknown
Instance,,,,,,,,,,,,,,,,,,,,,
111994,21.0,153,391652,1,1992.0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
111995,34.0,163,33653,0,1986.0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
111996,53.0,153,34765,1,1994.0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
111997,29.0,154,1494132,0,1984.0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
111998,17.0,191,120661,0,2007.0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [46]:
train_df.head()

,Age,Body Height [cm],Income in EUR,Size of City,Wears Glasses,Year of Record,pro_.net developer,pro_.net software developer,pro_Accountant,pro_Accounts Assistant,...,ud_Bachelor,ud_Master,ud_No,ud_PhD,h_c_0,h_c_Black,h_c_Blond,h_c_Brown,h_c_Red,h_c_Unknown
Instance,,,,,,,,,,,,,,,,,,,,,
2,41.0,186,91001.32764,1603504,0,1996.0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
3,28.0,170,157982.17670,1298017,1,2018.0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
4,33.0,171,45993.75793,751903,1,2006.0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
5,46.0,188,38022.16217,95389,0,2010.0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
6,60.0,181,125809.99030,56080,0,1991.0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0


In [47]:
test_df.shape

(73230, 1538)

In [48]:
train_df.shape

(111269, 1539)

In [49]:
train_df.head()

,Age,Body Height [cm],Income in EUR,Size of City,Wears Glasses,Year of Record,pro_.net developer,pro_.net software developer,pro_Accountant,pro_Accounts Assistant,...,ud_Bachelor,ud_Master,ud_No,ud_PhD,h_c_0,h_c_Black,h_c_Blond,h_c_Brown,h_c_Red,h_c_Unknown
Instance,,,,,,,,,,,,,,,,,,,,,
2,41.0,186,91001.32764,1603504,0,1996.0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
3,28.0,170,157982.17670,1298017,1,2018.0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
4,33.0,171,45993.75793,751903,1,2006.0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
5,46.0,188,38022.16217,95389,0,2010.0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
6,60.0,181,125809.99030,56080,0,1991.0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0


In [50]:
test_df.head()

,Age,Body Height [cm],Size of City,Wears Glasses,Year of Record,pro_.net developer,pro_.net software developer,pro_Accountant,pro_Accounts Assistant,pro_Accounts Clerk,...,ud_Bachelor,ud_Master,ud_No,ud_PhD,h_c_0,h_c_Black,h_c_Blond,h_c_Brown,h_c_Red,h_c_Unknown
Instance,,,,,,,,,,,,,,,,,,,,,
111994,21.0,153,391652,1,1992.0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
111995,34.0,163,33653,0,1986.0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
111996,53.0,153,34765,1,1994.0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
111997,29.0,154,1494132,0,1984.0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
111998,17.0,191,120661,0,2007.0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [51]:
#Split the dataframe into data and labels
X_train = train_df.drop('Income in EUR', axis=1) #data
y_train = train_df['Income in EUR'] #labels

In [52]:
X_train.shape
y_train.shape

(111269,)

In [53]:
X_train.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in X_train.columns.values]
#y_train.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in y_train.columns.values]

In [112]:
X1_train, X_test, y1_train, y_test = train_test_split(X_train, y_train, test_size=0.3, random_state=1234)

In [55]:
#Shape of the data
X_train.shape

(111269, 1538)

In [57]:
#X.head()

In [58]:
#Shape of the Labels
y_train.shape

(111269,)

In [123]:
model = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.05, max_depth = 5, alpha = 10, n_estimators = 100)
#model = xgb.XGBRegressor()
model.fit(X1_train,y1_train)

XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.3, gamma=0,
             importance_type='gain', learning_rate=0.05, max_delta_step=0,
             max_depth=5, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)

In [124]:
pred = model.predict(X_test)

In [125]:
rms = sqrt(mean_squared_error(y_test, pred))
print(rms)

127904.68636810067


In [ ]:
lab_enc = preprocessing.LabelEncoder()
training_scores_encoded = lab_enc.fit_transform(y)

In [64]:
#regr_rf = RandomForestRegressor(n_estimators = 700, random_state = 0)
#model_rf = regr_rf.fit(X,y)
acc = (model.score(X1_train,y1_train))
#model = LinearRegression().fit(X,y)
#acc = round(model.score(X, y))

In [65]:
acc

0.020507545762630808

In [66]:
missing(my_test_data,columns=my_test_data.columns)

missing vals
{'Instance': 0, 'Year of Record': 0, 'Gender': 0, 'Age': 0, 'Country': 0, 'Size of City': 0, 'Profession': 0, 'University Degree': 0, 'Wears Glasses': 0, 'Hair Color': 0, 'Body Height [cm]': 0, 'Income': 73230, 'train': 0}


In [67]:
my_test_data.shape

(73230, 13)

In [68]:
my_test_data['Hair Color'].fillna(my_test_data['Hair Color'].mode()[0], inplace=True)

In [69]:
my_test_data.shape

(73230, 13)

In [70]:
my_test_data['Gender'].fillna(my_test_data['Gender'].mode()[0], inplace=True)

In [71]:
missing(my_test_data, columns=my_test_data.columns)

missing vals
{'Instance': 0, 'Year of Record': 0, 'Gender': 0, 'Age': 0, 'Country': 0, 'Size of City': 0, 'Profession': 0, 'University Degree': 0, 'Wears Glasses': 0, 'Hair Color': 0, 'Body Height [cm]': 0, 'Income': 73230, 'train': 0}


In [72]:
my_test_data['Year of Record'].fillna(my_test_data['Year of Record'].mode()[0], inplace=True)

In [73]:
my_test_data.shape

(73230, 13)

In [74]:
my_test_data['Age'].fillna(my_test_data['Age'].mode()[0], inplace=True)

In [75]:
my_test_data['Profession'].fillna(my_test_data['Profession'].mode()[0], inplace=True)

In [76]:
my_test_data['University Degree'].fillna(my_data['University Degree'].mode()[0], inplace=True)

In [77]:
#my_test_data['Hair Color'].fillna(0, inplace=True)

In [78]:
missing(my_test_data, columns=my_test_data.columns)

missing vals
{'Instance': 0, 'Year of Record': 0, 'Gender': 0, 'Age': 0, 'Country': 0, 'Size of City': 0, 'Profession': 0, 'University Degree': 0, 'Wears Glasses': 0, 'Hair Color': 0, 'Body Height [cm]': 0, 'Income': 73230, 'train': 0}


In [79]:
my_test_data.shape

(73230, 13)

In [80]:
df_testing_data = pd.DataFrame()

In [81]:
my_test_data.shape

(73230, 13)

In [82]:
df_testing_data = my_test_data

In [83]:
df_testing_data.head()

,Instance,Year of Record,Gender,Age,Country,Size of City,Profession,University Degree,Wears Glasses,Hair Color,Body Height [cm],Income,train
0,111994,1992.0,other,21.0,Honduras,391652,senior project analyst,Master,1,Brown,153,NaN,0
1,111995,1986.0,other,34.0,Kyrgyzstan,33653,greeter,Bachelor,0,Black,163,NaN,0
2,111996,1994.0,unknown,53.0,Portugal,34765,liaison,Bachelor,1,Blond,153,NaN,0
3,111997,1984.0,0,29.0,Uruguay,1494132,occupational therapist,No,0,Black,154,NaN,0
4,111998,2007.0,other,17.0,Serbia,120661,portfolio manager,No,0,Red,191,NaN,0


In [84]:
# One-hot encode non binary variables
df_professiontest_oneHot = pd.get_dummies(df_testing_data['Profession'],
                                     prefix='profession')
df_gendertest_oneHot = pd.get_dummies(df_testing_data['Gender'],
                                     prefix='gender')
df_countrytest_oneHot = pd.get_dummies(df_testing_data['Country'],
                                     prefix='country')
df_udtest_oneHot = pd.get_dummies(df_testing_data['University Degree'],
                                     prefix='UD')
df_hctest_oneHot = pd.get_dummies(df_testing_data['Hair Color'],
                                     prefix='hc')

In [85]:
df_testing_data_enc = pd.DataFrame()

In [86]:
df_testing_data.shape

(73230, 13)

In [87]:
df_testing_data_enc = pd.concat([df_testing_data,
                              df_professiontest_oneHot,df_gendertest_oneHot,df_countrytest_oneHot,df_udtest_oneHot,df_hctest_oneHot
                             ], axis=1 )

In [88]:
df_testing_data_enc.shape

(73230, 1511)

In [89]:
df_testing_data_enc.drop(['Profession','Gender','Country','University Degree','Hair Color'], axis=1,inplace=True)

In [90]:
#del df_testing_data_enc['Hair Color']

In [91]:
df_testing_data_enc.shape

(73230, 1506)

In [102]:
df_testing_data_enc.rename(index = {"Body Height[cm]":
                     "Body Height cm"}, 
                                 inplace = True) 

In [103]:
df_testing_data_enc.head()

,Year of Record,Age,Size of City,Wears Glasses,Body Height _cm_,pro_.net developer,pro_.net software developer,pro_Accountant,pro_Accounts Assistant,pro_Accounts Clerk,...,ud_Bachelor,ud_Master,ud_No,ud_PhD,h_c_0,h_c_Black,h_c_Blond,h_c_Brown,h_c_Red,h_c_Unknown
0,1992.0,21.0,391652,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1986.0,34.0,33653,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1994.0,53.0,34765,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1984.0,29.0,1494132,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2007.0,17.0,120661,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [93]:
 df_testing_data_enc.shape

(73230, 1506)

In [95]:
for col in df_testing_data_enc.columns:
    if col not in X1_train.columns:
        print(col)
        df_testing_data_enc.drop(col, axis=1, inplace=True)

Instance
Body Height [cm]
Income
train
profession_.net developer
profession_.net software developer
profession_Accountant
profession_Accounts Assistant
profession_Accounts Clerk
profession_Accounts Manager
profession_Accounts Staff
profession_Acoustic Engineer
profession_Actor
profession_Actress
profession_Actuary
profession_Acupuncturist
profession_Adjustor
profession_Administration Assistant
profession_Administration Clerk
profession_Administration Manager
profession_Administration Staff
profession_Administrator
profession_Advertising Agent
profession_Advertising Assistant
profession_Advertising Clerk
profession_Advertising Contractor
profession_Advertising Executive
profession_Advertising Manager
profession_Advertising Staff
profession_Aerial Erector
profession_Aerobic Instructor
profession_Aeronautical Engineer
profession_Agent
profession_Air Traffic Controller
profession_Aircraft Designer
profession_Aircraft Engineer
profession_Aircraft Maintenance Engineer
profession_Aircraft Sur

profession_chief operating officer
profession_chief plan examiner
profession_chief review officer
profession_child care worker
profession_child welfare analyst
profession_chiropractor
profession_choreographer
profession_city assessor
profession_city laborer
profession_city medical examiner
profession_city park worker
profession_city planner
profession_city planning technician
profession_city research scientist
profession_civil design lead
profession_civil engineer
profession_civilian investigator
profession_claim specialist
profession_claims adjuster
profession_claims processor
profession_claims specialist
profession_clergy
profession_clerical aide
profession_clerical assistant
profession_clerical associate
profession_clerical supervisor
profession_clerk
profession_client services representative
profession_climber & pruner
profession_clinical case supervisor
profession_clinical director
profession_coach
profession_collections administrator
profession_collector
profession_college aide
p

profession_food server
profession_food service manager
profession_forensic logistics specialist
profession_forensic pathology technician
profession_forester
profession_foster care program evaluator
profession_front end developer
profession_functional analyst
profession_funeral director
profession_funeral service worker
profession_furniture coordinator
profession_furniture finisher
profession_gaming manager
profession_gaming surveillance officer
profession_gardener
profession_general counsel
profession_general manager 
profession_general services associate
profession_geological engineer
profession_geological technician 
profession_geologist
profession_geoscientist
profession_gis analyst
profession_gis editor
profession_gis specialist
profession_glazier
profession_grader
profession_grant coordinator
profession_grant manager
profession_grant project manager
profession_graphic artist
profession_green infrastructure engineer
profession_greeter
profession_grounds maintenance worker
professio

profession_nutritionist
profession_occupational therapist
profession_occupational therapist assistant
profession_office administrator
profession_office assistant
profession_office clerk
profession_office coordinator
profession_office machine aide
profession_office machine repairer
profession_office manager
profession_oil burner specialist
profession_ombudsperson
profession_on-line medical control physician
profession_open data lead analyst
profession_open market order coordinator
profession_operations associate
profession_operations coordinator
profession_operations customer service liaison
profession_operations desktop support technician
profession_operations manager
profession_optical network engineer
profession_optician
profession_optometrist
profession_order clerk
profession_order filler
profession_ordered deductions analyst
profession_ordnance handling expert
profession_outbreak data analyst
profession_outreach assistant
profession_outreach intern
profession_packager
profession_pa

profession_security guard
profession_security officer
profession_security specialist
profession_security system administrator
profession_security systems engineer
profession_senior accountant
profession_senior advisor
profession_senior analyst
profession_senior application engineer
profession_senior architect cloud infrastructure design and engineering
profession_senior bi developer
profession_senior budget analyst
profession_senior business analyst
profession_senior case support associate
profession_senior civil service advisor
profession_senior construction manager
profession_senior cook
profession_senior counsel
profession_senior crm developer
profession_senior data analyst
profession_senior data scientist
profession_senior design engineer
profession_senior developer
profession_senior director
profession_senior director it
profession_senior early childhood education consultant
profession_senior electrical engineer
profession_senior engineer
profession_senior estimator
profession_sen

profession_waitress
profession_water resources analyst
profession_water use inspector
profession_waterfront facilities engineer
profession_waterfront project manager
profession_watershed maintainer
profession_web application developer
profession_webform team member
profession_weigher
profession_welder
profession_wellness advocate
profession_wellness advocate supervisor
profession_windows administrator
profession_wireless coordinator
profession_woodworker
profession_word processor 
profession_workforce management analyst
profession_workforce planning intern
profession_writer
profession_x-ray technician
profession_yardmaster
profession_youth initiatives lead advisor
gender_0
gender_female
gender_male
gender_other
gender_unknown
country_Afghanistan
country_Albania
country_Algeria
country_Angola
country_Armenia
country_Australia
country_Austria
country_Azerbaijan
country_Bahamas
country_Bahrain
country_Belarus
country_Belgium
country_Belize
country_Benin
country_Bhutan
country_Bolivia
coun

In [96]:
for i in X1_train.columns:
    if i not in df_testing_data_enc.columns:
        print(i)
        df_testing_data_enc[i]= 0

Body Height _cm_
pro_.net developer
pro_.net software developer
pro_Accountant
pro_Accounts Assistant
pro_Accounts Clerk
pro_Accounts Manager
pro_Accounts Staff
pro_Acoustic Engineer
pro_Actor
pro_Actress
pro_Actuary
pro_Acupuncturist
pro_Adjustor
pro_Administration Assistant
pro_Administration Clerk
pro_Administration Manager
pro_Administration Staff
pro_Administrator
pro_Advertising Agent
pro_Advertising Assistant
pro_Advertising Clerk
pro_Advertising Contractor
pro_Advertising Executive
pro_Advertising Manager
pro_Advertising Staff
pro_Aerial Erector
pro_Aerobic Instructor
pro_Aeronautical Engineer
pro_Agent
pro_Air Traffic Controller
pro_Aircraft Designer
pro_Aircraft Engineer
pro_Aircraft Maintenance Engineer
pro_Aircraft Surface Finisher
pro_Airman
pro_Airport Controller
pro_Airport Manager
pro_Almoner
pro_Ambulance Controller
pro_Ambulance Crew
pro_Ambulance Driver
pro_Amusement Arcade Worker
pro_Anaesthetist
pro_Analyst
pro_Analytical Chemist
pro_Animal Breeder
pro_Anthropologi

pro_correctional officer
pro_correctional officers
pro_cosmetologist
pro_cost estimating manager
pro_cost estimator
pro_counselor
pro_courier
pro_court liaison officer
pro_court representative
pro_cranes & derricks inspector
pro_credit analyst
pro_credit authorizer
pro_crime analyst
pro_criminal investigator
pro_criminalist
pro_crm developer
pro_crossing guard
pro_cultural affairs coordinator
pro_curator
pro_custodial assistant
pro_custodian
pro_customer service
pro_customer service & operations analysis intern
pro_customer service representative
pro_cybersecurity operations analyst
pro_dam safety coordinator
pro_dancer
pro_data analyst
pro_data and process analyst
pro_data and technology analyst
pro_data architect
pro_data assistant
pro_data engineer
pro_data entry clerk
pro_data entry keyer
pro_data management assistant
pro_data manager
pro_data research analyst
pro_data research specialist
pro_data scientist
pro_data support analyst
pro_data warehouse architect
pro_database administ

pro_medical assistant
pro_medical examiner assistant
pro_medical records and health information technician
pro_medical scientist
pro_medical technician
pro_medicolegal analyst
pro_medicolegal investigator
pro_meeting and convention planner
pro_mental health coordinator
pro_messenger
pro_metal work mechanic
pro_meter reader
pro_metrics and reporting engineer
pro_middle school teacher
pro_millwright
pro_mining engineer
pro_mining machine operator
pro_mobile device management architect
pro_molder
pro_motor vehicle operator
pro_museum technician
pro_musician
pro_natural sciences manager
pro_naval architect
pro_neigborhood resiliency specialist
pro_network administrator
pro_network engineer
pro_network field technician
pro_network services manager
pro_news analyst
pro_news and street vendor
pro_nuclear engineer
pro_nuclear technicians
pro_nurse home visitor
pro_nursing supervisor
pro_nutrition consultant
pro_nutritionist
pro_occupational therapist
pro_occupational therapist assistant
pro_of

pro_software developer
pro_solar energy project manager
pro_solutions director
pro_sorter
pro_space analyst
pro_space scientists
pro_special assistant
pro_special assistant to assistant deputy commissioner
pro_special consultant
pro_special education teacher
pro_special examiner
pro_special projects manager
pro_special underwriting project manager
pro_speech-language pathologist
pro_spending analyst
pro_sql/oracle database administrator
pro_sr. analyst
pro_sr. internal auditor
pro_staff analyst
pro_staff analyst 2
pro_staff analyst ii
pro_staff analyst level ii
pro_staff attorney
pro_staff auditor
pro_staff counsel
pro_staff engineer / architect
pro_staff photographer
pro_standards specialist
pro_stationary engineer
pro_statistical analyst
pro_statistical assistant
pro_statistician
pro_steamfitter
pro_steel workers
pro_stock clerk
pro_stock worker level ii
pro_stonemason
pro_storage engineer
pro_stormwater program coordinator
pro_strategic account manager
pro_strategic initiatives coor

In [ ]:
#X.shape

In [97]:
df_testing_data_enc.shape

(73230, 1538)

In [104]:
predictions = model.predict(test_df)

ValueError: feature_names may not contain [, ] or <

In [ ]:
predictions[:20]

In [90]:
X.shape

(111269, 1538)

In [91]:
my_test_data.head()

,Instance,Year of Record,Gender,Age,Country,Size of City,Profession,University Degree,Wears Glasses,Hair Color,Body Height [cm],Income,train
0,111994,1992.0,other,21.0,Honduras,391652,senior project analyst,Master,1,Brown,153,NaN,0
1,111995,1986.0,other,34.0,Kyrgyzstan,33653,greeter,Bachelor,0,Black,163,NaN,0
2,111996,1994.0,unknown,53.0,Portugal,34765,liaison,Bachelor,1,Blond,153,NaN,0
3,111997,1984.0,0,29.0,Uruguay,1494132,occupational therapist,No,0,Black,154,NaN,0
4,111998,2007.0,other,17.0,Serbia,120661,portfolio manager,No,0,Red,191,NaN,0


In [92]:
y_true = predicted_income['Income']

In [93]:
y_pred = predictions

In [94]:
y_pred.shape

(73230,)

In [95]:
y_true.shape

(73230,)

In [96]:
my_test_data.shape

(73230, 13)

In [97]:
rms = sqrt(mean_squared_error(y_true, y_pred))

In [98]:
print(rms)

508191.34973698494


In [99]:
df = pd.DataFrame()

In [100]:
predictions.shape

(73230,)

In [101]:
predicted_income.shape

(73230, 2)

In [102]:
df_new = pd.DataFrame()

In [103]:
df_new['Instance'] = predicted_income['Instance']
# df['Income'] = predictions

In [104]:
df_new.shape

(73230, 1)

In [105]:
df_new['Income'] = predictions

In [106]:
df_new.head()

,Instance,Income
0,111994,24969.535692
1,111995,20567.015012
2,111996,34259.623322
3,111997,130957.765523
4,111998,33183.669309


In [107]:
df_new.to_csv('./ankitanew_4.csv',index=False)

In [108]:
df_new.head()

,Instance,Income
0,111994,24969.535692
1,111995,20567.015012
2,111996,34259.623322
3,111997,130957.765523
4,111998,33183.669309
